In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt


# Helper function to put some text on an axes to help identify them
def annotate_axes(ax, text, fontsize=18):
    ax.text(0.5, 0.5, text, transform=ax.transAxes,
            ha="center", va="center", fontsize=fontsize, color="darkgrey")

### Underlying tools

Underlying `subplots` and `subplot_mosaic` are the concept of a `ridSpec` and
a `SubplotSpec`:

`GridSpec`
    Specifies the geometry of the grid that a subplot will be
    placed. The number of rows and number of columns of the grid
    need to be set. Optionally, the subplot layout parameters
    (e.g., left, right, etc.) can be tuned.

`SubplotSpec`
    Specifies the location of the subplot in the given `GridSpec`.

### Adding single Axes at a time

The above functions create all Axes in a single function call.  It is also
possible to add Axes one at a time, and this was originally how Matplotlib
used to work.  Doing so is generally less elegant and flexible, though
sometimes useful for interactive work or to place an Axes in a custom
location:

`Figure.add_axes`
    Adds a single axes at a location specified by
    ``[left, bottom, width, height]`` in fractions of figure width or height.

`pyplot.subplot` or `Figure.add_subplot`
    Adds a single subplot on a figure, with 1-based indexing (inherited from
    Matlab).  Columns and rows can be spanned by specifying a range of grid
    cells.

`pyplot.subplot2grid`
    Similar to `.pyplot.subplot`, but uses 0-based indexing and two-d python
    slicing to choose cells.


## Low-level and advanced grid methods

Internally, the arrangement of a grid of Axes is controlled by creating
instances of `GridSpec` and `SubplotSpec`. *GridSpec* defines a
(possibly non-uniform) grid of cells. Indexing into the *GridSpec* returns
a SubplotSpec that covers one or more grid cells, and can be used to
specify the location of an Axes.

The following examples show how to use low-level methods to arrange Axes
using *GridSpec* objects.

### Basic 2x2 grid

We can accomplish a 2x2 grid in the same manner as
``plt.subplots(2, 2)``:



In [ ]:
fig = plt.figure(figsize=(5.5, 3.5), layout="constrained")
spec = fig.add_gridspec(ncols=2, nrows=2)

ax0 = fig.add_subplot(spec[0, 0])
annotate_axes(ax0, 'ax0')

ax1 = fig.add_subplot(spec[0, 1])
annotate_axes(ax1, 'ax1')

ax2 = fig.add_subplot(spec[1, 0])
annotate_axes(ax2, 'ax2')

ax3 = fig.add_subplot(spec[1, 1])
annotate_axes(ax3, 'ax3')

fig.suptitle('Manually added subplots using add_gridspec')

### Axes spanning rows or grids in a grid

We can index the *spec* array using [NumPy slice syntax](https://numpy.org/doc/stable/reference/arrays.indexing.html)
and the new Axes will span the slice.  This would be the same
as ``fig, axd = plt.subplot_mosaic([['ax0', 'ax0'], ['ax1', 'ax2']], ...)``:



In [ ]:
fig = plt.figure(figsize=(5.5, 3.5), layout="constrained")
spec = fig.add_gridspec(2, 2)

ax0 = fig.add_subplot(spec[0, :])
annotate_axes(ax0, 'ax0')

ax10 = fig.add_subplot(spec[1, 0])
annotate_axes(ax10, 'ax10')

ax11 = fig.add_subplot(spec[1, 1])
annotate_axes(ax11, 'ax11')

fig.suptitle('Manually added subplots, spanning a column')

### Manual adjustments to a *GridSpec* layout

When a  *GridSpec* is explicitly used, you can adjust the layout
parameters of subplots that are created from the  *GridSpec*.  Note this
option is not compatible with ``constrained_layout`` or
`Figure.tight_layout` which both ignore *left* and *right* and adjust
subplot sizes to fill the figure.  Usually such manual placement
requires iterations to make the Axes tick labels not overlap the Axes.

These spacing parameters can also be passed to `pyplot.subplots` and
`pyplot.subplot_mosaic` as the *gridspec_kw* argument.



In [ ]:
fig = plt.figure(layout=None, facecolor='0.9')
gs = fig.add_gridspec(nrows=3, ncols=3, left=0.05, right=0.75,
                      hspace=0.1, wspace=0.05)
ax0 = fig.add_subplot(gs[:-1, :])
annotate_axes(ax0, 'ax0')
ax1 = fig.add_subplot(gs[-1, :-1])
annotate_axes(ax1, 'ax1')
ax2 = fig.add_subplot(gs[-1, -1])
annotate_axes(ax2, 'ax2')
fig.suptitle('Manual gridspec with right=0.75')

### Nested layouts with SubplotSpec

You can create nested layout similar to `Figure.subfigures` using
`SubplotSpec.subgridspec`.  Here the Axes spines *are*
aligned.

Note this is also available from the more verbose
`GridSpecFromSubplotSpec`.



In [ ]:
fig = plt.figure(layout="constrained")
gs0 = fig.add_gridspec(1, 2)

gs00 = gs0[0].subgridspec(2, 2)
gs01 = gs0[1].subgridspec(3, 1)

for a in range(2):
    for b in range(2):
        ax = fig.add_subplot(gs00[a, b])
        annotate_axes(ax, f'axLeft[{a}, {b}]', fontsize=10)
        if a == 1 and b == 1:
            ax.set_xlabel('xlabel')
for a in range(3):
    ax = fig.add_subplot(gs01[a])
    annotate_axes(ax, f'axRight[{a}, {b}]')
    if a == 2:
        ax.set_ylabel('ylabel')

fig.suptitle('nested gridspecs')

Here's a more sophisticated example of nested *GridSpec*: We create an outer
4x4 grid with each cell containing an inner 3x3 grid of Axes. We outline
the outer 4x4 grid by hiding appropriate spines in each of the inner 3x3
grids.



In [ ]:
def squiggle_xy(a, b, c, d, i=np.arange(0.0, 2*np.pi, 0.05)):
    return np.sin(i*a)*np.cos(i*b), np.sin(i*c)*np.cos(i*d)

fig = plt.figure(figsize=(8, 8), constrained_layout=False)
outer_grid = fig.add_gridspec(4, 4, wspace=0, hspace=0)

for a in range(4):
    for b in range(4):
        # gridspec inside gridspec
        inner_grid = outer_grid[a, b].subgridspec(3, 3, wspace=0, hspace=0)
        axs = inner_grid.subplots()  # Create all subplots for the inner grid.
        for (c, d), ax in np.ndenumerate(axs):
            ax.plot(*squiggle_xy(a + 1, b + 1, c + 1, d + 1))
            ax.set(xticks=[], yticks=[])

# show only the outside spines
for ax in fig.get_axes():
    ss = ax.get_subplotspec()
    ax.spines.top.set_visible(ss.is_first_row())
    ax.spines.bottom.set_visible(ss.is_last_row())
    ax.spines.left.set_visible(ss.is_first_col())
    ax.spines.right.set_visible(ss.is_last_col())

plt.show()

### Resources
- https://matplotlib.org/stable/tutorials/intermediate/arranging_axes.html#low-level-and-advanced-grid-methods
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/axes_demo.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/ganged_plots.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/gridspec_and_subplots.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/gridspec_multicolumn.html
- https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_adjust.html